# Phase-2.4: Hybrid Temporal Curation (v3 - Panel-Safe)

## Mission

**CRITICAL FIX:** Previous curation (uniform random sampling) achieved 0% temporal correlation.  
This notebook implements **Hybrid Temporal + Local Clustering** to preserve:

- ✅ Temporal attack patterns (multi-stage attacks)
- ✅ Local neighborhood structure (similarity detection)
- ✅ Feature distribution (statistical completeness)
- ✅ Rare variant coverage (zero-day capability)

## Algorithm

For each attack type:
1. **Temporal Windowing:** Divide chronologically into 100 buckets
2. **Local Clustering:** Within each bucket, cluster into 250 groups (MiniBatchKMeans)
3. **Medoid Selection:** Select cluster medoid (closest to centroid)
4. **Metadata Preservation:** Store original_index, temporal_bucket, cluster_id

**Target:** ~25,000 representatives per attack type (250 samples × 100 buckets)

**Expected Validation:**
- Temporal correlation: ≥85% (vs 0% before)
- NN stability: ≥75% (vs 55% before)
- KL divergence: <0.10 (vs NaN before)
- Recall@K: ≥92% (maintained)

## 🔧 Setup & Imports

In [1]:
import numpy as np
import pandas as pd
import chromadb
from chromadb.config import Settings
from sklearn.cluster import MiniBatchKMeans
from sklearn.metrics import pairwise_distances_argmin_min
from tqdm.auto import tqdm
import os
import time
from pathlib import Path
import gc

print("✅ Imports complete")
print(f"📦 NumPy version: {np.__version__}")
print(f"📦 Pandas version: {pd.__version__}")
print(f"📦 ChromaDB version: {chromadb.__version__}")

✅ Imports complete
📦 NumPy version: 2.1.3
📦 Pandas version: 2.2.3
📦 ChromaDB version: 1.4.1


## 🎯 Intel GPU Acceleration Check

**Context:** User has Intel Arc iGPU.  
**Options:** sklearn-intelex, Intel Extension for Scikit-learn

**Reality Check:** MiniBatchKMeans CPU implementation is already optimized.  
Intel extensions primarily benefit matrix operations (SVD, PCA), not k-means.

**Decision:** Use CPU-optimized sklearn (no GPU acceleration needed for laptop feasibility).

In [2]:
# Intel GPU Acceleration Investigation
print("🔍 Checking Intel GPU acceleration options...\n")

intel_acceleration = False
acceleration_method = "None (CPU-optimized sklearn)"

try:
    # Check if sklearn-intelex is available
    from sklearnex import patch_sklearn, unpatch_sklearn
    
    # Patch sklearn to use Intel optimizations
    patch_sklearn()
    
    intel_acceleration = True
    acceleration_method = "Intel Extension for Scikit-learn (sklearnex)"
    print("✅ Intel Extension for Scikit-learn detected")
    print("   → MiniBatchKMeans will use Intel optimizations")
    print("   → Expected speedup: 1.5-3× on Intel CPUs/iGPUs\n")
    
except ImportError:
    print("ℹ️  Intel Extension for Scikit-learn NOT installed")
    print("   → Using standard CPU-optimized sklearn")
    print("   → Still laptop-feasible (~5-10 minutes)")
    print("\n💡 To enable Intel acceleration (optional):")
    print("   pip install scikit-learn-intelex")
    print("   → Provides 1.5-3× speedup on Intel hardware\n")

print(f"📊 Acceleration method: {acceleration_method}")
print(f"🎯 Target runtime: {'3-5 minutes' if intel_acceleration else '5-10 minutes'}")

🔍 Checking Intel GPU acceleration options...

ℹ️  Intel Extension for Scikit-learn NOT installed
   → Using standard CPU-optimized sklearn
   → Still laptop-feasible (~5-10 minutes)

💡 To enable Intel acceleration (optional):
   pip install scikit-learn-intelex
   → Provides 1.5-3× speedup on Intel hardware

📊 Acceleration method: None (CPU-optimized sklearn)
🎯 Target runtime: 5-10 minutes


## 📂 Configuration

In [3]:
# Paths
BASE_DIR = Path(r"c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection")
PHASE_2_DIR = BASE_DIR / "artifacts" / "phase_2"
CHROMADB_DIR = BASE_DIR / "artifacts" / "chromadb"

# Data files (CORRECTED)
VECTORS_PATH = PHASE_2_DIR / "feature_vectors.npy"
METADATA_PATH = PHASE_2_DIR / "vector_metadata.parquet"  # Using parquet for efficiency

# ChromaDB configuration
COLLECTION_NAME = "iot_behavioral_memory_hybrid"

# Hybrid curation parameters
HYBRID_CONFIG = {
    'temporal_buckets': 100,          # 100 time windows per attack type
    'clusters_per_bucket': 250,       # 250 clusters per time window
    'target_samples_per_attack': 25000,  # Total budget per attack type
    'min_bucket_size': 100,           # Minimum samples in bucket to cluster
    'rare_variant_boost': 1.5,        # Oversample rare variants (not implemented yet)
    'preserve_temporal_index': True,  # CRITICAL: Store original indices
    'random_state': 42
}

# Memory optimization
BATCH_SIZE = 5000  # ChromaDB batch size

print("✅ Configuration loaded")
print(f"\n📊 Hybrid Curation Parameters:")
for key, value in HYBRID_CONFIG.items():
    print(f"   • {key}: {value}")

print(f"\n📁 Data paths:")
print(f"   • Vectors: {VECTORS_PATH}")
print(f"   • Metadata: {METADATA_PATH}")
print(f"   • ChromaDB: {CHROMADB_DIR}")

# Verify files exist
if not VECTORS_PATH.exists():
    print(f"\n❌ ERROR: Vectors file not found: {VECTORS_PATH}")
    print(f"   Available .npy files in {PHASE_2_DIR}:")
    for file in PHASE_2_DIR.glob("*.npy"):
        print(f"      • {file.name}")
else:
    print(f"\n✅ Vectors file exists")

if not METADATA_PATH.exists():
    print(f"\n❌ ERROR: Metadata file not found: {METADATA_PATH}")
    print(f"   Available .parquet files in {PHASE_2_DIR}:")
    for file in PHASE_2_DIR.glob("*.parquet"):
        print(f"      • {file.name}")
else:
    print(f"✅ Metadata file exists")

✅ Configuration loaded

📊 Hybrid Curation Parameters:
   • temporal_buckets: 100
   • clusters_per_bucket: 250
   • target_samples_per_attack: 25000
   • min_bucket_size: 100
   • rare_variant_boost: 1.5
   • preserve_temporal_index: True
   • random_state: 42

📁 Data paths:
   • Vectors: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\artifacts\phase_2\feature_vectors.npy
   • Metadata: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\artifacts\phase_2\vector_metadata.parquet
   • ChromaDB: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\artifacts\chromadb

✅ Vectors file exists
✅ Metadata file exists


In [4]:
# 🧹 CHROMADB PRE-EXECUTION CHECK
print("🧹 Checking ChromaDB state before execution...\n")
print("="*80)

# Initialize temporary client to check existing collections
temp_client = chromadb.PersistentClient(
    path=str(CHROMADB_DIR),
    settings=Settings(anonymized_telemetry=False, allow_reset=True)
)

# List all existing collections
existing_collections = temp_client.list_collections()
print(f"📊 Existing Collections: {len(existing_collections)}")

if existing_collections:
    for coll in existing_collections:
        print(f"   • {coll.name}: {coll.count():,} vectors")
    
    # Check if target collection already exists
    if COLLECTION_NAME in [c.name for c in existing_collections]:
        print(f"\n⚠️  Collection '{COLLECTION_NAME}' already exists")
        print(f"   → Will be DELETED and recreated (preventing duplicates)")
    else:
        print(f"\n✅ Collection '{COLLECTION_NAME}' does not exist yet")
else:
    print("   → No existing collections (fresh start)")

print(f"\n🎯 Target collection: {COLLECTION_NAME}")
print(f"✅ Pre-execution check complete")
print("="*80)

🧹 Checking ChromaDB state before execution...

📊 Existing Collections: 2
   • iot_behavioral_memory_hybrid: 457,622 vectors
   • iot_behavioral_memory_curated: 502,630 vectors

⚠️  Collection 'iot_behavioral_memory_hybrid' already exists
   → Will be DELETED and recreated (preventing duplicates)

🎯 Target collection: iot_behavioral_memory_hybrid
✅ Pre-execution check complete


## 📥 Load Full Dataset

In [5]:
print("📥 Loading full dataset...\n")
print("="*80)

# Load vectors (memory-mapped for efficiency)
print("Loading vectors (memory-mapped)...")
full_vectors = np.load(VECTORS_PATH, mmap_mode='r')
print(f"✅ Vectors shape: {full_vectors.shape}")
print(f"   • Samples: {full_vectors.shape[0]:,}")
print(f"   • Features: {full_vectors.shape[1]}")
print(f"   • Dtype: {full_vectors.dtype}")

# Load metadata (using parquet for efficiency)
print("\nLoading metadata...")
full_metadata = pd.read_parquet(METADATA_PATH)
print(f"✅ Metadata shape: {full_metadata.shape}")
print(f"   • Rows: {len(full_metadata):,}")
print(f"   • Columns: {full_metadata.columns.tolist()}")

# Verify alignment
assert len(full_vectors) == len(full_metadata), "Vector-metadata mismatch!"
print(f"\n✅ Vector-metadata alignment verified")

# Attack type distribution
print(f"\n📊 Attack Type Distribution:")
type_counts = full_metadata['type'].value_counts()
for attack_type, count in type_counts.items():
    percentage = count / len(full_metadata) * 100
    print(f"   • {attack_type:15s}: {count:10,} ({percentage:5.2f}%)")

print("="*80)

📥 Loading full dataset...

Loading vectors (memory-mapped)...
✅ Vectors shape: (22339021, 99)
   • Samples: 22,339,021
   • Features: 99
   • Dtype: float32

Loading metadata...
✅ Metadata shape: (22339021, 3)
   • Rows: 22,339,021
   • Columns: ['label', 'type', 'vector_id']

✅ Vector-metadata alignment verified

📊 Attack Type Distribution:
   • scanning       :  7,140,161 (31.96%)
   • ddos           :  6,165,008 (27.60%)
   • dos            :  3,375,328 (15.11%)
   • xss            :  2,108,944 ( 9.44%)
   • password       :  1,718,568 ( 7.69%)
   • normal         :    796,380 ( 3.56%)
   • backdoor       :    508,116 ( 2.27%)
   • injection      :    452,659 ( 2.03%)
   • ransomware     :     72,805 ( 0.33%)
   • mitm           :      1,052 ( 0.00%)


## 🔬 Hybrid Temporal Curation Algorithm

### Core Algorithm:

```python
for each attack_type:
    # Step 1: Temporal windowing
    chronological_indices = sort_by_temporal_order(attack_data)
    buckets = divide_into_N_windows(chronological_indices, N=100)
    
    # Step 2: Local clustering per bucket
    for each bucket:
        if bucket_size >= min_size:
            clusters = MiniBatchKMeans(n_clusters=250).fit(bucket_vectors)
            
            # Step 3: Select medoid per cluster
            for each cluster:
                medoid = find_closest_to_centroid(cluster_vectors)
                
                # Step 4: Store with metadata
                curated_samples.append({
                    'vector': medoid_vector,
                    'original_index': medoid_index,  # CRITICAL
                    'temporal_bucket': bucket_id,
                    'cluster_id': cluster_id
                })
```

In [6]:
def hybrid_temporal_curation(vectors, metadata, config):
    """
    Hybrid Temporal + Local Clustering Curation
    
    Args:
        vectors: Memory-mapped numpy array (N, D)
        metadata: DataFrame with 'type' and temporal ordering
        config: Hybrid configuration dictionary
    
    Returns:
        curated_vectors: List of selected vectors
        curated_metadata: List of metadata dicts
    """
    print("🔬 Starting Hybrid Temporal Curation...\n")
    print("="*80)
    
    curated_vectors = []
    curated_metadata = []
    
    # Get attack types (exclude normal)
    attack_types = [t for t in metadata['type'].unique() if t != 'normal']
    
    # Add normal separately with simple stratified sampling
    attack_types_with_normal = attack_types + ['normal']
    
    print(f"📊 Attack types to process: {len(attack_types_with_normal)}")
    print(f"   • Attacks: {attack_types}")
    print(f"   • Normal traffic: Separate processing\n")
    
    total_start_time = time.time()
    
    for attack_type in tqdm(attack_types_with_normal, desc="Processing attack types"):
        print(f"\n{'='*80}")
        print(f"🎯 Processing: {attack_type}")
        print(f"{'='*80}")
        
        # Get attack data
        attack_mask = metadata['type'] == attack_type
        attack_indices = metadata[attack_mask].index.tolist()
        attack_metadata = metadata[attack_mask]
        
        print(f"   • Total samples: {len(attack_indices):,}")
        
        # Special handling for normal (use simple temporal stratification)
        if attack_type == 'normal':
            target_samples = 275000  # Match previous curation
            print(f"   • Target samples: {target_samples:,}")
            print(f"   • Method: Simple temporal stratification (no clustering)\n")
            
            # Temporal stratification without clustering
            n_buckets = config['temporal_buckets']
            samples_per_bucket = target_samples // n_buckets
            
            bucket_edges = np.linspace(0, len(attack_indices), n_buckets + 1, dtype=int)
            
            for bucket_idx in range(n_buckets):
                start_idx = bucket_edges[bucket_idx]
                end_idx = bucket_edges[bucket_idx + 1]
                
                bucket_indices = attack_indices[start_idx:end_idx]
                
                if len(bucket_indices) == 0:
                    continue
                
                # Randomly sample from bucket
                n_samples = min(samples_per_bucket, len(bucket_indices))
                sampled_indices = np.random.choice(bucket_indices, n_samples, replace=False)
                
                # Store selected samples
                for orig_idx in sampled_indices:
                    curated_vectors.append(vectors[orig_idx])
                    curated_metadata.append({
                        'type': attack_type,
                        'original_index': int(orig_idx),
                        'temporal_bucket': bucket_idx,
                        'cluster_id': -1,  # No clustering for normal
                        'curation_method': 'temporal_stratified'
                    })
            
            print(f"   ✅ Selected {len([m for m in curated_metadata if m['type'] == attack_type]):,} samples")
            continue
        
        # Hybrid temporal + clustering for attacks
        target_samples = config['target_samples_per_attack']
        n_buckets = config['temporal_buckets']
        n_clusters = config['clusters_per_bucket']
        
        print(f"   • Target samples: {target_samples:,}")
        print(f"   • Temporal buckets: {n_buckets}")
        print(f"   • Clusters per bucket: {n_clusters}")
        print(f"   • Expected samples per bucket: {target_samples // n_buckets}\n")
        
        # Create temporal buckets (chronological order preserved by index)
        bucket_edges = np.linspace(0, len(attack_indices), n_buckets + 1, dtype=int)
        
        bucket_samples = []
        
        for bucket_idx in tqdm(range(n_buckets), desc=f"  Temporal buckets", leave=False):
            start_idx = bucket_edges[bucket_idx]
            end_idx = bucket_edges[bucket_idx + 1]
            
            bucket_indices = attack_indices[start_idx:end_idx]
            
            if len(bucket_indices) < config['min_bucket_size']:
                # Bucket too small, sample all
                for orig_idx in bucket_indices:
                    bucket_samples.append({
                        'vector': vectors[orig_idx],
                        'original_index': int(orig_idx),
                        'temporal_bucket': bucket_idx,
                        'cluster_id': -1,  # No clustering
                        'type': attack_type
                    })
                continue
            
            # Load bucket vectors into memory (convert to float64 for clustering)
            bucket_vectors = np.array([vectors[i] for i in bucket_indices], dtype=np.float64)
            
            # Adaptive clustering: Use fewer clusters if bucket is small
            actual_n_clusters = min(n_clusters, len(bucket_indices) // 10)
            
            if actual_n_clusters < 2:
                # Too small to cluster, sample all
                for i, orig_idx in enumerate(bucket_indices):
                    bucket_samples.append({
                        'vector': bucket_vectors[i],
                        'original_index': int(orig_idx),
                        'temporal_bucket': bucket_idx,
                        'cluster_id': -1,
                        'type': attack_type
                    })
                continue
            
            # Cluster within temporal bucket
            try:
                kmeans = MiniBatchKMeans(
                    n_clusters=actual_n_clusters,
                    random_state=config['random_state'],
                    batch_size=min(1000, len(bucket_vectors)),
                    max_iter=100,
                    n_init=3
                )
                cluster_labels = kmeans.fit_predict(bucket_vectors)
                cluster_centers = kmeans.cluster_centers_
                
                # Select medoid (closest to centroid) per cluster
                for cluster_id in range(actual_n_clusters):
                    cluster_mask = cluster_labels == cluster_id
                    cluster_vectors = bucket_vectors[cluster_mask]
                    cluster_original_indices = [bucket_indices[i] for i, mask in enumerate(cluster_mask) if mask]
                    
                    if len(cluster_vectors) == 0:
                        continue
                    
                    # Find medoid (closest to centroid)
                    centroid = cluster_centers[cluster_id].reshape(1, -1)
                    medoid_idx_in_cluster = pairwise_distances_argmin_min(centroid, cluster_vectors)[0][0]
                    medoid_original_idx = cluster_original_indices[medoid_idx_in_cluster]
                    
                    bucket_samples.append({
                        'vector': cluster_vectors[medoid_idx_in_cluster],
                        'original_index': int(medoid_original_idx),
                        'temporal_bucket': bucket_idx,
                        'cluster_id': cluster_id,
                        'type': attack_type
                    })
                
            except Exception as e:
                print(f"\n⚠️  Clustering failed for bucket {bucket_idx}: {e}")
                print(f"    Falling back to random sampling...")
                # Fallback: random sample
                n_samples = min(n_clusters, len(bucket_indices))
                sampled_idx = np.random.choice(len(bucket_indices), n_samples, replace=False)
                for i in sampled_idx:
                    bucket_samples.append({
                        'vector': bucket_vectors[i],
                        'original_index': int(bucket_indices[i]),
                        'temporal_bucket': bucket_idx,
                        'cluster_id': -1,
                        'type': attack_type
                    })
        
        # Add to curated dataset
        for sample in bucket_samples:
            curated_vectors.append(sample['vector'])
            curated_metadata.append({
                'type': sample['type'],
                'original_index': sample['original_index'],
                'temporal_bucket': sample['temporal_bucket'],
                'cluster_id': sample['cluster_id'],
                'curation_method': 'hybrid_temporal_clustering'
            })
        
        print(f"   ✅ Selected {len(bucket_samples):,} samples (target: {target_samples:,})")
        
        # Free memory
        del bucket_samples
        gc.collect()
    
    total_time = time.time() - total_start_time
    
    print(f"\n{'='*80}")
    print(f"✅ Hybrid Temporal Curation Complete")
    print(f"{'='*80}")
    print(f"   • Total curated samples: {len(curated_vectors):,}")
    print(f"   • Total execution time: {total_time:.2f} seconds ({total_time/60:.2f} minutes)")
    print(f"   • Samples per second: {len(curated_vectors)/total_time:.0f}")
    
    return curated_vectors, curated_metadata

print("✅ Hybrid curation function defined")

✅ Hybrid curation function defined


## 🚀 Execute Hybrid Curation

In [7]:
# Run hybrid curation
curated_vectors, curated_metadata = hybrid_temporal_curation(
    full_vectors,
    full_metadata,
    HYBRID_CONFIG
)

# Convert to numpy array
curated_vectors_array = np.array(curated_vectors, dtype=np.float64)
curated_metadata_df = pd.DataFrame(curated_metadata)

print(f"\n📊 Curated Dataset Summary:")
print(f"   • Vectors shape: {curated_vectors_array.shape}")
print(f"   • Vectors dtype: {curated_vectors_array.dtype}")
print(f"   • Metadata rows: {len(curated_metadata_df):,}")
print(f"   • Compression ratio: {len(full_metadata)/len(curated_metadata_df):.1f}×")

print(f"\n📊 Attack Type Distribution (Curated):")
curated_type_counts = curated_metadata_df['type'].value_counts()
for attack_type, count in curated_type_counts.items():
    percentage = count / len(curated_metadata_df) * 100
    print(f"   • {attack_type:15s}: {count:7,} ({percentage:5.2f}%)")

🔬 Starting Hybrid Temporal Curation...

📊 Attack types to process: 10
   • Attacks: ['scanning', 'dos', 'injection', 'ddos', 'password', 'xss', 'ransomware', 'backdoor', 'mitm']
   • Normal traffic: Separate processing



Processing attack types:   0%|          | 0/10 [00:00<?, ?it/s]


🎯 Processing: scanning
   • Total samples: 7,140,161
   • Target samples: 25,000
   • Temporal buckets: 100
   • Clusters per bucket: 250
   • Expected samples per bucket: 250



  Temporal buckets:   0%|          | 0/100 [00:00<?, ?it/s]

   ✅ Selected 24,988 samples (target: 25,000)

🎯 Processing: dos
   • Total samples: 3,375,328
   • Target samples: 25,000
   • Temporal buckets: 100
   • Clusters per bucket: 250
   • Expected samples per bucket: 250



...

   ✅ Selected 6,894 samples (target: 25,000)

🎯 Processing: backdoor
   • Total samples: 508,116
   • Target samples: 25,000
   • Temporal buckets: 100
   • Clusters per bucket: 250
   • Expected samples per bucket: 250



  Temporal buckets:   0%|          | 0/100 [00:00<?, ?it/s]

   ✅ Selected 24,998 samples (target: 25,000)

🎯 Processing: mitm
   • Total samples: 1,052
   • Target samples: 25,000
   • Temporal buckets: 100
   • Clusters per bucket: 250
   • Expected samples per bucket: 250



  Temporal buckets:   0%|          | 0/100 [00:00<?, ?it/s]

   ✅ Selected 1,052 samples (target: 25,000)

🎯 Processing: normal
   • Total samples: 796,380
   • Target samples: 275,000
   • Method: Simple temporal stratification (no clustering)

   ✅ Selected 275,000 samples

✅ Hybrid Temporal Curation Complete
   • Total curated samples: 457,622
   • Total execution time: 1514.54 seconds (25.24 minutes)
   • Samples per second: 302

📊 Curated Dataset Summary:
   • Vectors shape: (457622, 99)
   • Vectors dtype: float64
   • Metadata rows: 457,622
   • Compression ratio: 48.8×

📊 Attack Type Distribution (Curated):
   • normal         : 275,000 (60.09%)
   • backdoor       :  24,998 ( 5.46%)
   • scanning       :  24,988 ( 5.46%)
   • injection      :  24,985 ( 5.46%)
   • xss            :  24,961 ( 5.45%)
   • password       :  24,957 ( 5.45%)
   • dos            :  24,894 ( 5.44%)
   • ddos           :  24,893 ( 5.44%)
   • ransomware     :   6,894 ( 1.51%)
   • mitm           :   1,052 ( 0.23%)


## 💾 Build ChromaDB Collection

In [8]:
print("💾 Building ChromaDB collection...\n")
print("="*80)

# Initialize ChromaDB client
chroma_settings = Settings(
    anonymized_telemetry=False,
    allow_reset=True
)

client = chromadb.PersistentClient(
    path=str(CHROMADB_DIR),
    settings=chroma_settings
)

print(f"✅ ChromaDB client initialized")
print(f"   • Path: {CHROMADB_DIR}")

# Delete existing collection if exists
try:
    client.delete_collection(COLLECTION_NAME)
    print(f"   • Deleted existing collection: {COLLECTION_NAME}")
except:
    pass

# Create new collection with COSINE distance metric
collection = client.create_collection(
    name=COLLECTION_NAME,
    metadata={
        "hnsw:space": "cosine",  # CRITICAL: Use cosine distance for pattern-based similarity
        "description": "Hybrid Temporal + Local Clustering Curation (v3 - Panel-Safe)",
        "curation_method": "hybrid_temporal_clustering",
        "temporal_buckets": HYBRID_CONFIG['temporal_buckets'],
        "clusters_per_bucket": HYBRID_CONFIG['clusters_per_bucket'],
        "total_samples": len(curated_vectors_array),
        "compression_ratio": float(len(full_metadata) / len(curated_metadata_df))
    }
)

print(f"✅ Collection created: {COLLECTION_NAME}")
print(f"\n📤 Adding vectors to collection (batch size: {BATCH_SIZE})...\n")

# Add in batches
n_vectors = len(curated_vectors_array)
n_batches = (n_vectors + BATCH_SIZE - 1) // BATCH_SIZE

for batch_idx in tqdm(range(n_batches), desc="Uploading batches"):
    start_idx = batch_idx * BATCH_SIZE
    end_idx = min(start_idx + BATCH_SIZE, n_vectors)
    
    batch_vectors = curated_vectors_array[start_idx:end_idx].tolist()
    batch_metadata = curated_metadata_df.iloc[start_idx:end_idx].to_dict('records')
    batch_ids = [f"hybrid_{i}" for i in range(start_idx, end_idx)]
    
    collection.add(
        embeddings=batch_vectors,
        metadatas=batch_metadata,
        ids=batch_ids
    )

# Verify
collection_count = collection.count()
print(f"\n✅ ChromaDB collection built successfully")
print(f"   • Collection name: {COLLECTION_NAME}")
print(f"   • Total vectors: {collection_count:,}")
print(f"   • Expected: {n_vectors:,}")
print(f"   • Match: {'✅ YES' if collection_count == n_vectors else '❌ NO'}")

print("="*80)

💾 Building ChromaDB collection...

✅ ChromaDB client initialized
   • Path: c:\Users\suhas\OneDrive\Desktop\Capstone\RAG-IDS-Knowledge-Augmented-IoT-Threat-Detection\artifacts\chromadb
   • Deleted existing collection: iot_behavioral_memory_hybrid
✅ Collection created: iot_behavioral_memory_hybrid

📤 Adding vectors to collection (batch size: 5000)...



Uploading batches:   0%|          | 0/92 [00:00<?, ?it/s]


✅ ChromaDB collection built successfully
   • Collection name: iot_behavioral_memory_hybrid
   • Total vectors: 457,622
   • Expected: 457,622
   • Match: ✅ YES


## 📊 Curation Summary Statistics

In [9]:
print("📊 Curation Summary Statistics\n")
print("="*80)

# Overall statistics
print("\n📈 Overall Metrics:")
print(f"   • Full dataset size: {len(full_metadata):,} vectors")
print(f"   • Curated dataset size: {len(curated_metadata_df):,} vectors")
print(f"   • Compression ratio: {len(full_metadata)/len(curated_metadata_df):.2f}×")
print(f"   • Sampling rate: {len(curated_metadata_df)/len(full_metadata)*100:.2f}%")

# Per-attack-type statistics
print("\n📊 Per-Attack-Type Statistics:")
print(f"\n{'Attack Type':<15} {'Full':<12} {'Curated':<10} {'Ratio':<8} {'Method':<30}")
print("-" * 80)

for attack_type in full_metadata['type'].unique():
    full_count = len(full_metadata[full_metadata['type'] == attack_type])
    curated_count = len(curated_metadata_df[curated_metadata_df['type'] == attack_type])
    ratio = full_count / curated_count if curated_count > 0 else 0
    
    # Determine method
    if attack_type == 'normal':
        method = "Temporal Stratified"
    else:
        method = "Hybrid Temporal + Clustering"
    
    print(f"{attack_type:<15} {full_count:<12,} {curated_count:<10,} {ratio:<8.1f}× {method:<30}")

# Temporal distribution check
print("\n📅 Temporal Bucket Distribution (Attack Types Only):")
attack_curated = curated_metadata_df[curated_metadata_df['type'] != 'normal']
bucket_counts = attack_curated.groupby(['type', 'temporal_bucket']).size().groupby('type').describe()
print(bucket_counts[['mean', 'std', 'min', 'max']])

# Clustering statistics
print("\n🔬 Clustering Statistics (Attack Types Only):")
clustered = attack_curated[attack_curated['cluster_id'] >= 0]
print(f"   • Samples with clustering: {len(clustered):,} ({len(clustered)/len(attack_curated)*100:.1f}%)")
print(f"   • Samples without clustering: {len(attack_curated) - len(clustered):,} (small buckets)")

if len(clustered) > 0:
    clusters_per_type = clustered.groupby('type')['cluster_id'].nunique()
    print(f"\n   Clusters per attack type:")
    for attack_type, n_clusters in clusters_per_type.items():
        print(f"      • {attack_type}: {n_clusters} clusters")

print("\n" + "="*80)
print("✅ Hybrid Temporal Curation Complete")
print("="*80)

📊 Curation Summary Statistics


📈 Overall Metrics:
   • Full dataset size: 22,339,021 vectors
   • Curated dataset size: 457,622 vectors
   • Compression ratio: 48.82×
   • Sampling rate: 2.05%

📊 Per-Attack-Type Statistics:

Attack Type     Full         Curated    Ratio    Method                        
--------------------------------------------------------------------------------
normal          796,380      275,000    2.9     × Temporal Stratified           
scanning        7,140,161    24,988     285.7   × Hybrid Temporal + Clustering  
dos             3,375,328    24,894     135.6   × Hybrid Temporal + Clustering  
injection       452,659      24,985     18.1    × Hybrid Temporal + Clustering  
ddos            6,165,008    24,893     247.7   × Hybrid Temporal + Clustering  
password        1,718,568    24,957     68.9    × Hybrid Temporal + Clustering  
xss             2,108,944    24,961     84.5    × Hybrid Temporal + Clustering  
ransomware      72,805       6,894      10.6  

## 🎯 Next Steps

1. **Run Validation**: Execute `Phase_2_6_Hybrid_Curation_Validation.ipynb`
2. **Expected Results**:
   - ✅ Temporal correlation: ≥85% (vs 0% before)
   - ✅ NN stability: ≥75% (vs 55% before)
   - ✅ KL divergence: <0.10 (vs NaN before)
   - ✅ Recall@K: ≥92% (maintained)
   - ✅ Entropy: ≥99% (maintained)
   - ✅ Rare variants: ≥98% (maintained)

3. **If validation passes (≥5/6 checks)**: Proceed to Phase-3 (RAG Pipeline)
4. **If validation fails**: Adjust `HYBRID_CONFIG` parameters and re-run